## Загрузка датасета на HuggingFace

Этот ноутбук дает пример того, как залить локальный датасет на ХФ. Адаптируйте его под свой датасет. Затем, выложите в гитхаб получившийся ноутбук (приложите к своему датасету), чтобы всегда был доступен код для заливки вашего датасета на ХФ. Убедитесь, что ячейки последовательно запускаются.

In [ ]:
from PIL import Image
import json
import datasets
from tqdm import tqdm
import os

### Подготовка данных

#### WARNING! 

Если ваш датасет является __ПРИВАТНЫМ__, то оставьте `MY_DATASET_IS_PRIVATE_LETS_HIDE_ANSWERS` равным `True`. Иначе, поставьте `False`. Этот флаг дальше используется, чтобы стереть ответы перед загрузкой на ХФ датасета. На ХФ даже приватно не должно лежать датасетов с ответами!

In [ ]:
MY_DATASET_IS_PRIVATE_LETS_HIDE_ANSWERS = False

Данный пример рассчитан на загрузку на ХФ локального датасета.

Параметр `path_to_data` - это путь ДО файлов `shots.json` и `test.json`, которые вы будете дальше загружать на ХФ в виде датасета или домена датасета. 

Параметр `path_to_meta` - это путь ДО меты датасета.

Итоговые пути будут собираться из `path_to_data` / `path_to_meta` + `file_name.json`!

In [ ]:
path_to_data = "./"
path_to_meta = "./"

Сплиты и мета лежат в формате JSON.

In [ ]:
def load_json(path):
    with open(path) as f:
        data = json.load(f)
    return data

#### Подгрузка данных

Считайте сплиты и мету датасета (домена датасета). Это просто JSON файлики либо внутри прямо папки датасета, либо внутри папки по названию домена, который вы будете загружать.

In [ ]:
shots = load_json(os.path.join(path_to_data, "shots.json"))["data"]
test = load_json(os.path.join(path_to_data, "test.json"))["data"]
meta = load_json(os.path.join(path_to_meta, "dataset_meta.json"))

Из меты для датасета нужны только промпты.

In [ ]:
prompts = meta["prompts"]

#### Обработка полей датасета

На ХФ вы загружаете датасет, где у КАЖДОГО сэмпла вместо числа в поле instruction стоит промпт. Число указывает, какой по индексу взять промпт из секции с промптами в мете датасета.

In [ ]:
for card in shots:
    card["instruction"] = prompts[card["instruction"]]

for card in test:
    card["instruction"] = prompts[card["instruction"]]

Теперь вам нужно "обработать" вашу модальность(-и). Если у вас в датасете картинки, то для каждой вы вместо пути к картинке подгружаете саму картинку и убираете у нее `filename`.

Зачем убирать `filename`? Если этого не сделать, то потом при конвертации в байты модуль datasets увидит, что картинка взята по какому-то пути и не будет ее конвертировать в байты. А нам нужно превратить PIL.Image в байткод и загрузить на ХФ именно его.

Для картинок вы можете воспользоваться функцией `convert_images`. Она принимает на вход: 
- `dataset_split`: список словарей, это список сэмплов сплита (json файлик, который вы загрузили ранее)
- `feature_name`: строка, название поля, которое преобразуется (например, "image", "image_1")
- `path_to_samples`: строка. Если ваши коллеги для картинок указывали не полный путь до них, а только название файлика, то, чтобы открыть файлик, вам придется указать, как попасть в папку samples, где и находятся картинки. Тогда, укажите в `path_to_samples` путь от папки, где у вас этот ноутбук до папки `samples` (включая ее в путь)

In [ ]:
def convert_images(dataset_split: list[dict], feature_name: str, path_to_samples: str = None):
    for card in tqdm(dataset_split):
        if path_to_samples is None:
            path_to_image = card["inputs"][feature_name]
        else:
            path_to_image = os.path.join(path_to_samples, card["inputs"][feature_name])
        
        card["inputs"][feature_name] = Image.open(path_to_image)
        card["inputs"][feature_name].filename = ""

In [ ]:
convert_images(shots, "image")
convert_images(test, "image")

Для аудио модальности файл ничем не открываем, а просто сохраняем ссылки на аудио-данные. Обработка аудио-данных будет сделана позднее с помощью datasets.Audio() при формировании features.

In [ ]:
def convert_audio(dataset_split: list[dict], feature_name: str, path_to_samples: str = None):
    for card in tqdm(dataset_split):
        if path_to_samples is None:
            path_to_audio = card["inputs"][feature_name]
        else:
            path_to_audio = os.path.join(path_to_samples, card["inputs"][feature_name])

        card["inputs"][feature_name] = path_to_audio

convert_audio(shots, "audio")
convert_audio(test, "audio")

#### Убираем ответы для приватных задач

Надеемся, вы поставили в начале ноутбука корректное значение `MY_DATASET_IS_PRIVATE_LETS_HIDE_ANSWERS`.

Если там стоит `True`, то в `test` сплите ответы на все задания стираются. Вместо них остается пустая строка, чтобы вы случайно не пушнули на ХФ датасет с заполненными ответами, и они не утекли.

In [ ]:
def hide_answers(dataset_split: list[dict]):
    for card in tqdm(dataset_split):
        card["outputs"] = ""

In [ ]:
if MY_DATASET_IS_PRIVATE_LETS_HIDE_ANSWERS:
    hide_answers(test)

### Создаем датасет для загрузки на ХФ

#### Аннотация полей датасета

В `features` повторяется структура КАЖДОГО сэмпла вашего датасета с описанием формата данных в каждом поле. 
- `instruction` всегда строка
- `meta` - id всегда целое число

Далее смотрите по тому, какие поля у вашего датасета.

`features` нужен для того, чтобы ХФ сам автоматически создал техническую часть README.md датасета, заполнив ее информацией, которая используется при загрузке датасета. Отсутствие `features` может и обычно приводит к невозможности использовать датасет. Ровно такие же последствия будут от ошибок в заполнении (например, неправильно указан тип данных).

__Внимание!__ Если у вас в датасете в разных вопросах разное количество ответов, то поля в `features` нужно заполнить для сэмпла с НАИБОЛЬШИМ количеством ответов. Иначе говоря, представьте, что у вас у всех вопросов в датасете максимальное количество вариантов ответа, просто некоторые пустые. Вот из такого соображения и заполняйте `features`. Он один на весь датасет и должен охватывать все поля, которые в нем встречаются!

In [ ]:
features = datasets.Features({
    "instruction": datasets.Value("string"),
    "inputs": {
        "image": datasets.Image(decode=False),
        "question": datasets.Value("string"),
    },
    "outputs": datasets.Value("string"),
    "meta": {
        "id": datasets.Value("int32"),
        "domain": datasets.Value("string"),
        'image': {
            'synt_source': datasets.Sequence([datasets.Value("string")]),
            'source': datasets.Value("string"),
            'type': datasets.Value("string"),
            'content': datasets.Value("string"),
            'context': datasets.Value("string")
        }
    },
})


Пример features для аудио:

In [ ]:
features = datasets.Features({
    "instruction": datasets.Value("string"),
    "inputs": {
        "audio": datasets.Audio(),
        "question": datasets.Value("string"),
    },
    "outputs": datasets.Value("string"),
    "meta": {
        "id": datasets.Value("int32"),
        "domain": datasets.Value("string"),
        'audio': {
            'synt_source': datasets.Sequence([datasets.Value("string")]),
            'source': datasets.Value("string"),
            'type': datasets.Value("string"),
            'content': datasets.Value("string"),
            'context': datasets.Value("string")
        }
    },
})

#### Создание датасетов для каждого сплита

Теперь создаем сплиты датасета. Можно это сделать либо в одну строку:

In [ ]:
shots_ds = datasets.Dataset.from_list(shots, features=features)

Но это способ для маленьких датасетов. Большие датасеты так создаются крайне долго. Чтобы побыстрее собрать большой датасет, можно разбить его на кусочки по N сэмплов. Перегонять каждый кусочек и присоединять к уже конвертированным ранее кусочкам.

In [ ]:
STEP = 20

lst_steps = []
for i in tqdm(range(0, len(test), STEP)):
    tmp = datasets.Dataset.from_list(test[i: i+STEP], features=features)
    lst_steps.extend([tmp])
    
test_ds = datasets.concatenate_datasets(lst_steps)

Для аудио-данных, чтобы выделилсь нужные фичи, следует дополнительно сделать (для картинок этот код можно исполнить, ничего сломаться не должно):

In [ ]:
shots_ds = shots_ds.cast(features)
test_ds = test_ds.cast(features)

##### Проверка

Если вы собирали датасет по кускам, то разумно будет проверить, что сборка прошла успешно - ничего не потеряно, не продублировано и так далее.

Но вы можете проверить целостность датасета даже, если и не по кусочкам собирали его. Так вы можете отловить ошибки до того, как их найдут на ревью :)

In [ ]:
# проверка, что id вопросов сходятся

bools = []
for i in range(len(test)):
    bools.extend([test[i]["meta"]["id"] == test_ds[i]["meta"]["id"]])
all(bools)

In [ ]:
# проверка, что количество вопросов до конвертации и после осталось одинаковым

len(test) == len(test_ds)

#### Собираем сплиты в один датасет

In [ ]:
dataset = datasets.DatasetDict({"shots": shots_ds, "test": test_ds})

### Загрузка датасета на ХФ

Для загрузки на ХФ вам понадобятся:
- Токен. Это строка, содержащая ключик, который позволит вам записывать в репозиторий. 
- Путь для записи. Это тоже строка, которая содержит путь, по которому вы выложите свой датасет. Этот путь содержит название аккаунта (MERA-evaluation) и название вашего датасета. Название датасета пишите ровно так, как оно заявлено в мете! Регистр тоже имеет значение!

Советуем опубликовывать сперва всё приватно, и выслать на почту mera@a-ai.ru токен и путь для верификации. 
Если ваш сет публичный и вы хотите отправить всё публично, то в Merge request просто пришлите путь к сету.

In [ ]:
### TOKEN
token = "token"
###

### UPLOAD PATH
dataset_path_hub = "MERA-evaluation/ruCLEVR"
###


# Если вы хотите предварительно протестировать, как датасет будет выглядеть после заливки на ХФ,
# то можно загрузить его сначала к себе в приватный репозиторий

### UPLOAD PATH
# dataset_path_hub = "artemorloff/ruclevr"
###

In [ ]:
dataset.push_to_hub(dataset_path_hub, private=True, token=token) # опубликовать приватно

### Проверка того, как датасет загрузился на ХФ

После загрузки датасета будет полезно посмотреть, как его будет видеть любой человек, который после вашей загрузки его скачает. 

Загрузите датасет целиком, используя `datasets.load_dataset(dataset_path_hub)`, а затем проверьте, что:
- все поля на месте. Если у вас в датасете у разных вопросов было разное количество вариантов ответа, то теперь их везде станет одинаковое количество. Недостающие варианты ответа у каждого вопроса теперь будут прописаны, но будут иметь значение `None`. Это нормально.
- ваша модальность корректно обработалась. Если у вас в датасете были картинки, то все они должны превратиться в байткод. Не должно остаться ни одной картинки, которая не конвертирована в байты. Если у картинки есть и байты, и путь прописан (а не `None`), то это окей. `bytes` точно должны быть заполнены, `path` может быть None.
- датасет идентичен по содержанию исходному. То есть, в исходном JSON и загруженном датасете вопрос с одинаковым `id` имеет одинаково заполненные поля (кроме тех, что заполняются `None`, как описано выше).

In [ ]:
ds = datasets.load_dataset(dataset_path_hub)

In [ ]:
ds

Пример проверки двух сплитов, что в них везде картинки конвертированы в байты

In [ ]:
check = []
for card in ds["shots"]:
    image_converted_to_bytes = isinstance(card["inputs"]["image"]["bytes"], bytes)
    check.extend([image_converted_to_bytes])

all(check)

In [ ]:
check = []
for card in ds["test"]:
    image_converted_to_bytes = isinstance(card["inputs"]["image"]["bytes"], bytes)
    check.extend([image_converted_to_bytes])

all(check)

Пример проверки двух сплитов, что в них тексты вопросов совпадают с оригинальными

In [ ]:
check = []
for idx, card in enumerate(ds["shots"]):
    same_question = shots[idx]["inputs"]["question"] == card["inputs"]["question"]
    check.extend([same_question])

all(check)

In [ ]:
check = []
for idx, card in enumerate(ds["test"]):
    same_question = test[idx]["inputs"]["question"] == card["inputs"]["question"]
    check.extend([same_question])

all(check)

Для аудио после загрузки данных с huggingface получаем следующую информацию:
- path - имя аудио-файла
- array - аудио-данные в виде массива array


- sampling_rate - частота дискретизации аудио

In [ ]:
import json
import datasets
from tqdm import tqdm
import os

dataset_path_hub = "rakalexandra/fake_audio"
ds = datasets.load_dataset(dataset_path_hub)
ds["shots"][0]["inputs"]["audio"]

Проверки для аудио, что все данные те же самые:
- path_to_samples - папка с первоначалными аудио, которые ранее грузили на hf

In [ ]:
import soundfile as sf
from tqdm import tqdm

def check_audio(hf_dataset_split: list[dict], dataset_split: list[dict], path_to_samples: str = None):
    assert len(dataset_split) == len(hf_dataset_split)
    for idx, item in tqdm(enumerate(hf_dataset_split)):
        orig_audio_name = dataset_split[idx]["inputs"]["audio"]

        sr = item["inputs"]["audio"]["sampling_rate"]
        audio_name = item["inputs"]["audio"]["path"]
        data = item["inputs"]["audio"]["array"]
        
        if path_to_samples is None:
            path_to_audio = orig_audio_name
        else:
            path_to_audio = os.path.join(path_to_samples, orig_audio_name)

        orig_audio, orig_sample_rate = sf.read(path_to_audio)

        assert orig_audio_name.split("/")[-1] == audio_name
        assert sr == orig_sample_rate
        assert all(data == orig_audio)

check_audio(ds["shots"], shots, "/path/to/orig_audio_set")
check_audio(ds["test"], test, "/path/to/orig_audio_set")

Если хотим сохранить аудио, загруженные с hf на диск:

In [ ]:
new_audio_path = "/path/to/audio"

for item in ds["shots"]:
    sr = item["inputs"]["audio"]["sampling_rate"]
    audio_name = item["inputs"]["audio"]["path"]
    data = item["inputs"]["audio"]["array"]
    sf.write(os.path.join(new_audio_path, audio_name), data, sr)